In [ ]:
pV_max = readdlm("pV_opt_value.txt", Float64)
pV_max = pV_max[:]
p0 = pV_max[1:gridap.np]
#p0 = zeros(gridap.np)
N = num_free_dofs(gridap.FE_U)
D_mat0 = MatrixD(gridap)
#α = [0. 1.0/(2*10.0)]
#control = ControllingParameters(flag_f,flag_t,r,β,η,α,nparts,nkx,K,Amp,Dp,pv,c,ηe,ηd)
LHp=[Inf h1+h2]
LHn=[Inf h3]
phys = PhysicalParameters(k,ω,ϵ1,ϵ2,ϵ3,ϵd,μ,R,σs,dpml,LHp,LHn,wg_center,wg_size)
nkx = 100
kx_ini = -π/L
dkx = 2*π/L/nkx
kx_end = π/L-dkx
kx_s = range(kx_ini,kx_end;length=nkx)
Q_mat = MatrixQt(phys.k,phys.ϵ1,gridap)

gk = zeros(nkx)
Giik = zeros(nkx)
pV_initial = zeros(gridap.np+2*N*control.K)
pV_initial[1:gridap.np]=p0[:]
pf = pf_p(p0;control,gridap)
if (control.flag_f)
    pfh = FEFunction(gridap.FE_Pf,pf)
else
    pfh = FEFunction(gridap.FE_P,pf)
end
ph = (pf->Threshold(pf;control))∘pfh
if (control.Dp)
    D_mat = MatrixDp(ph,gridap)
else
    D_mat = D_mat0
end
for ki=1:nkx
    kx=kx_s[ki]
    A_mat = MatrixA(kx,ph;phys,control,gridap)
    G_ii,V_raw,info = eigsolve(x->MatrixG(x;A_mat,D_mat,Q_mat),rand(ComplexF64,N),2,:LM;ishermitian=true)
    Giik[ki]=sum(G_ii)
    V_mat = rand(ComplexF64,N,control.K)
    for ib = 1:2
        V_mat[:,ib]=V_raw[ib]
    end
    V_mat = reinterpret(Float64,V_mat)
    pV_initial[gridap.np+1:end] = V_mat[:]
    gk[ki]=g_pV(pV_initial,[];kx,Q_mat,D_mat0,phys,control,gridap)
end
sum(gk)/nkx/control.Amp/L

In [ ]:
scene, layout = layoutscene(resolution=(1400/2,900/2))
ax = layout[1,1]=Axis(scene)
lin1 = scatter!(ax,kx_s[:]./(2*π/L),gk[:]./control.Amp/L,markersize=5)
lin2 = lines!(ax,kx_s[:]./(2*π/L),Giik[:]./L)
ax.ylabel="Power"
ax.xlabel="kxL/2π"
ax.title="Optimzed with L=0.6 and α=1/2000"
save("Results/L0.6_gvsk.png",scene)
scene

In [ ]:
scene, layout = layoutscene(resolution=(800/2,500/2))
ax = layout[1,1]=Axis(scene)
lin1 = scatter!(ax,range(1,length(G_iin),step=1),abs.(G_iin)/L,markersize=10)
#ax.xlabel="i'th eigenvalue"
#ax.ylabel="Value"
#ax.title="Power Eigenvalue Distribution"
#ax.xlabelsize=20
#ax.ylabelsize=20
#ax.titlesize=20
#axislegend(ax, [lin1, lin2],["Optimized","Random"], position = :lt,
#    orientation = :vertical)
scene